In [1]:
import os
import librosa
import numpy as np
import pandas as pd

# Path to audio files
audio_folder_path = r'AudioWAV'

# Prepare list to hold features and labels
features_list = []
emotions = []

# Define a dictionary for mapping emotion codes to full labels
emotion_dict = {
    'ANG': 'Anger',
    'DIS': 'Disgust',
    'FEA': 'Fear',
    'HAP': 'Happy',
    'NEU': 'Neutral',
    'SAD': 'Sad'
}

# Loop through each audio file in the folder
for filename in os.listdir(audio_folder_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(audio_folder_path, filename)
        
        # Load audio
        y, sr = librosa.load(file_path)
        
        # Define 20ms window size
        window_size_ms = 0.02  # 20ms
        frame_length = int(window_size_ms * sr)  # Samples for 20ms window
        hop_length = frame_length // 2  # 50% overlap
        
        # Extract MFCC
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length, n_fft=frame_length)
        
        # Extract delta and delta-delta MFCC
        delta_mfcc = librosa.feature.delta(mfcc)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        
        # Calculate statistics for MFCC, delta-MFCC, and delta-delta-MFCC
        mfcc_features = np.concatenate([
            np.mean(mfcc, axis=1), np.std(mfcc, axis=1)
        ])
        delta_mfcc_features = np.concatenate([
            np.mean(delta_mfcc, axis=1), np.std(delta_mfcc, axis=1)
        ])
        delta2_mfcc_features = np.concatenate([
            np.mean(delta2_mfcc, axis=1), np.std(delta2_mfcc, axis=1)
        ])
        
        # Combine all features
        features = np.concatenate([mfcc_features, delta_mfcc_features, delta2_mfcc_features])
        features_list.append(features)
        
        # Extract emotion labels from filename
        parts = filename.split('_')
        emotion_code = parts[2]  # Emotion code (e.g., ANG, DIS)
        
        # Map emotion code to full emotion label
        emotion_label = emotion_dict.get(emotion_code, "Unknown")
        emotions.append(emotion_label)

# Define column names for DataFrame
columns = []
columns += [f'mfcc_mean_{i}' for i in range(13)] + [f'mfcc_std_{i}' for i in range(13)]
columns += [f'delta_mfcc_mean_{i}' for i in range(13)] + [f'delta_mfcc_std_{i}' for i in range(13)]
columns += [f'delta2_mfcc_mean_{i}' for i in range(13)] + [f'delta2_mfcc_std_{i}' for i in range(13)]

# Create DataFrame with features and labels
df = pd.DataFrame(features_list, columns=columns)
df['emotion'] = emotions  # Add emotion labels

# Save to CSV
df.to_csv('audio_features_without_filtering.csv', index=False)
print("Features with labels have been extracted and saved to 'audio_features_without_filtering.csv'")


Features with labels have been extracted and saved to 'audio_features_without_filtering.csv'


In [2]:
df

,mfcc_mean_0,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,...,delta2_mfcc_std_4,delta2_mfcc_std_5,delta2_mfcc_std_6,delta2_mfcc_std_7,delta2_mfcc_std_8,delta2_mfcc_std_9,delta2_mfcc_std_10,delta2_mfcc_std_11,delta2_mfcc_std_12,emotion
0,-508.958221,118.634850,-3.356938,40.970490,3.092985,15.026862,-17.768225,-5.002903,-11.964508,-2.992107,...,1.868941,1.050583,1.215996,1.395098,1.106331,0.931807,1.114635,0.931815,1.174189,Anger
1,-556.846436,129.902359,-12.017046,52.574577,5.240597,24.657318,-22.256861,5.648675,-12.966894,-2.986880,...,1.694662,1.328769,1.243354,1.272423,0.964516,0.959910,1.023868,1.036770,1.011040,Disgust
2,-512.270203,106.605949,4.481588,33.221615,10.252079,11.566597,-15.322458,-2.085884,-8.047853,-5.309411,...,1.737946,1.255807,1.169335,1.337092,0.984128,0.957654,0.902779,0.929083,0.755707,Fear
3,-507.348145,120.728325,-9.146874,42.703300,7.847485,13.112681,-21.243057,-0.663292,-12.059789,-8.139682,...,1.628116,1.237468,1.284604,1.347286,1.266892,1.093934,1.267915,0.997715,1.104413,Happy
4,-538.608521,123.066727,-0.616907,40.799046,10.415956,14.509365,-13.737962,-3.017844,-11.575838,-6.111917,...,1.529162,1.156719,1.291078,1.373941,1.104398,1.017255,1.232664,1.270499,0.978076,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7437,-622.698975,127.347740,16.230566,54.328545,-11.143003,42.037441,-17.240387,12.382068,-12.402145,5.875704,...,1.516225,1.382194,1.144804,1.143540,1.030847,0.981285,0.950538,0.969847,1.156949,Disgust
7438,-633.114319,123.366188,16.944687,59.779652,-8.737967,43.901775,-17.874489,11.103239,-9.550373,5.951777,...,1.328429,1.144373,1.054839,1.203048,0.943827,0.941787,1.028106,0.868207,1.089416,Fear
7439,-581.527649,122.734200,12.707020,53.749996,-9.003664,29.595524,-20.057116,9.435844,-12.970961,5.856920,...,1.310597,1.508141,1.530938,1.222889,1.369629,1.065622,1.064884,0.956855,0.909332,Happy
7440,-598.897339,119.986755,23.080889,45.074135,-11.070361,30.927126,-11.557021,7.765401,-10.457713,6.325374,...,1.805060,1.358548,1.153678,1.357628,1.106659,0.998317,1.261371,0.988496,1.027129,Neutral


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data from CSV file
file_path = 'C:/Users/eswar/Desktop/Ml HW/Final Project/audio_features_without_filtering.csv'  # Replace with your local file path
data = pd.read_csv(file_path)

# Define features (X) and target (y) columns
X = data.drop(columns=['emotion'])  # Drop the target column to get features
y = data['emotion']  # Define the target column

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine X and y for each set and save them back to CSV
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Save to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Training and test data have been saved to 'training_data.csv' and 'test_data.csv'.")


Training and test data have been saved to 'training_data.csv' and 'test_data.csv'.


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Load the train and test CSV files with extracted features
train_df = pd.read_csv('C:/Users/eswar/Desktop/Ml HW/Final Project/train.csv')
test_df = pd.read_csv('C:/Users/eswar/Desktop/Ml HW/Final Project/test.csv')

# Drop the 'intensity' column if it exists
if 'intensity' in train_df.columns:
    train_df = train_df.drop(columns=['intensity'])

if 'intensity' in test_df.columns:
    test_df = test_df.drop(columns=['intensity'])

# Separate features and labels, assuming 'emotion' is the target label
if 'emotion' in train_df.columns:
    y_train = train_df['emotion'].reset_index(drop=True)
    train_features = train_df.drop(columns=['emotion']).reset_index(drop=True)
else:
    train_features = train_df.reset_index(drop=True)

if 'emotion' in test_df.columns:
    y_test = test_df['emotion'].reset_index(drop=True)
    test_features = test_df.drop(columns=['emotion']).reset_index(drop=True)
else:
    test_features = test_df.reset_index(drop=True)

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')
train_features = pd.DataFrame(imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(imputer.transform(test_features), columns=test_features.columns)

# Standardize the features
scaler = StandardScaler(with_mean=False)
train_features = pd.DataFrame(scaler.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns)

# Apply PCA to retain 95% of the explained variance
pca = PCA(n_components=0.95)
train_features_pca = pca.fit_transform(train_features)
test_features_pca = pca.transform(test_features)

# Convert PCA-transformed features back to DataFrame and reset index
train_features_pca_df = pd.DataFrame(train_features_pca).reset_index(drop=True)
test_features_pca_df = pd.DataFrame(test_features_pca).reset_index(drop=True)

# Add 'emotion' labels back to the transformed data if they were present
if 'emotion' in train_df.columns:
    train_features_pca_df['emotion'] = y_train
    test_features_pca_df['emotion'] = y_test

# Save PCA-transformed data back to CSV
train_features_pca_df.to_csv('train_pca.csv', index=False)
test_features_pca_df.to_csv('test_pca.csv', index=False)

print("PCA transformation complete.")
print("Training features shape after PCA:", train_features_pca_df.shape)
print("Testing features shape after PCA:", test_features_pca_df.shape)
print("Explained variance by selected components:", np.sum(pca.explained_variance_ratio_))


PCA transformation complete.
Training features shape after PCA: (5953, 50)
Testing features shape after PCA: (1489, 50)
Explained variance by selected components: 0.9530030097876494


In [6]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score

# Load PCA-transformed train and test data
X_train = pd.read_csv(r'C:/Users/eswar/Desktop/Ml HW/Final Project/train_pca.csv')
X_test = pd.read_csv(r'C:/Users/eswar/Desktop/Ml HW/Final Project/test_pca.csv')

# Separate target labels from features in training data
Y_train = X_train['emotion']
X_train = X_train.drop(columns=['emotion'], errors='ignore')  # Ensure only features in X_train
X_test = X_test.drop(columns=['emotion'], errors='ignore')    # Ensure only features in X_test

# Function to perform cross-validation
def cross_validate(X_train, Y_train, model_type="svm"):
    model = SVC() if model_type == "svm" else RandomForestClassifier(n_estimators=800)
    scores = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')
    print(f"Cross-Validation Scores ({model_type}):", scores)
    print(f"Mean Cross-Validation Score ({model_type}): {scores.mean()}")

# SVM classifier function
def svm_classifier(X_train, Y_train, X_test):
    svm = SVC()
    svm.fit(X_train, Y_train)
    return svm.predict(X_test)

# Perform cross-validation on PCA features
cross_validate(X_train, Y_train, model_type="svm")

# Classify PCA test features using the SVM classifier
y_test_svm = svm_classifier(X_train, Y_train, X_test)

# Save predictions
test_df = pd.DataFrame(X_test)  # Initialize with X_test to get correct row count
test_df['emotion'] = y_test_svm  # Add predictions
test_df.to_csv(r'C:/Users/eswar/Desktop/Ml HW/Final Project/predict_svm.csv', index=False)

print("Predictions saved to 'predict_svm.csv'")


Cross-Validation Scores (svm): [0.48026868 0.50461797 0.49622166 0.4789916  0.50672269]
Mean Cross-Validation Score (svm): 0.49336451961137107
Predictions saved to 'predict_svm.csv'
